# 📌 **01 - Pinecone: Base Vetorial**

## 🎯 **Objetivo:**
Configurar e popular o Pinecone com dados turísticos para Rio e Paris.

## 📋 **O que faremos:**
1. ⚙️ Setup do Pinecone
2. 📊 Criação do índice vetorial  
3. 📝 Indexação dos dados turísticos
4. ✅ Testes de conectividade

---

## 1️⃣ **Setup e Configuração**

In [ ]:
# Instalação das dependências
!pip install -q pinecone-client langchain-huggingface sentence-transformers langchain-groq python-dotenv

In [ ]:
# Imports necessários
from pinecone import Pinecone, ServerlessSpec
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
import time

# Carregar variáveis de ambiente
load_dotenv()

print("✅ Bibliotecas importadas com sucesso!")

## 2️⃣ **Configuração do Pinecone**

In [ ]:
# Configuração das credenciais
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT', 'us-east-1')
INDEX_NAME = "turismo-inteligente"

if not PINECONE_API_KEY:
    print("⚠️ Configure PINECONE_API_KEY no arquivo .env")
else:
    print("✅ Credenciais carregadas!")

In [ ]:
# Inicializar cliente Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Verificar índices existentes
print("📋 Índices existentes:")
for index in pc.list_indexes().names():
    print(f"  - {index}")

In [ ]:
# Criar índice se não existir
if INDEX_NAME not in pc.list_indexes().names():
    print(f"🔨 Criando índice '{INDEX_NAME}'...")
    
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,  # Para sentence-transformers/all-MiniLM-L6-v2
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region=PINECONE_ENVIRONMENT
        )
    )
    
    # Aguardar inicialização
    while not pc.describe_index(INDEX_NAME).status['ready']:
        time.sleep(1)
        print("⏳ Aguardando inicialização...")
    
    print("✅ Índice criado com sucesso!")
else:
    print(f"✅ Índice '{INDEX_NAME}' já existe!")

# Conectar ao índice
index = pc.Index(INDEX_NAME)
print(f"🔗 Conectado ao índice: {INDEX_NAME}")

## 3️⃣ **Setup dos Embeddings**

In [ ]:
# Inicializar modelo de embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

print("🧠 Modelo de embeddings carregado!")
print(f"📏 Dimensão: {len(embeddings.embed_query('teste'))}")

## 4️⃣ **Dados Turísticos**

In [ ]:
# Base de conhecimento turístico
dados_turismo = {
    "rio_roteiros": [
        "Cristo Redentor: Uma das Sete Maravilhas do Mundo Moderno, localizado no Corcovado.",
        "Pão de Açúcar: Bondinho famoso com vista panorâmica da cidade e praias.",
        "Praia de Copacabana: 4km de areia branca, calçadão famoso, hotéis luxuosos.",
        "Praia de Ipanema: Bairro sofisticado, praia das celebridades, pôr do sol inesquecível.",
        "Santa Teresa: Bairro boêmio, casarões coloniais, ateliês de artistas."
    ],
    "rio_logistica": [
        "Aeroporto Galeão: Principal aeroporto internacional, 20km do centro.",
        "Metro Rio: Linhas 1, 2 e 4 conectam principais pontos turísticos.",
        "Uber/99: Disponível 24h, mais seguro que táxi comum.",
        "BRT: Ônibus rápido conecta Barra da Tijuca ao centro.",
        "Hospedagem Copacabana: Hotéis de todas as categorias na orla."
    ],
    "paris_roteiros": [
        "Torre Eiffel: Símbolo de Paris, 324m de altura, iluminação noturna espetacular.",
        "Museu do Louvre: Maior museu do mundo, Mona Lisa, arte clássica.",
        "Notre-Dame: Catedral gótica, arquitetura medieval, Ile de la Cité.",
        "Champs-Élysées: Avenida mais famosa, compras, cafés, Arc de Triomphe.",
        "Montmartre: Bairro artístico, Sacré-Cœur, cabarés, vista panorâmica."
    ],
    "paris_logistica": [
        "Charles de Gaulle: Principal aeroporto, RER B conecta ao centro em 45min.",
        "Metro Paris: 14 linhas, passe Navigo, funciona até 1h15 (2h15 sexta/sábado).",
        "Uber/Taxi: Disponível, mas metro é mais rápido no centro.",
        "Hotel Le Marais: Bairro histórico, walking distance de atrações.",
        "Velib: Sistema de bikes compartilhadas, ideal para curtas distâncias."
    ]
}

print("📚 Base de conhecimento preparada!")
total_docs = sum(len(docs) for docs in dados_turismo.values())
print(f"📊 Total de documentos: {total_docs}")

## 5️⃣ **Indexação no Pinecone**

In [ ]:
# Verificar se já temos dados
stats = index.describe_index_stats()
print(f"📊 Vetores já indexados: {stats['total_vector_count']}")

if stats['total_vector_count'] == 0:
    print("🔄 Iniciando indexação...")
    
    vectors_to_upsert = []
    
    for categoria, documentos in dados_turismo.items():
        for i, doc in enumerate(documentos):
            # Gerar embedding
            embedding = embeddings.embed_query(doc)
            
            # Criar ID único
            vector_id = f"{categoria}_{i}"
            
            # Preparar vetor com metadados
            vector_data = {
                "id": vector_id,
                "values": embedding,
                "metadata": {
                    "text": doc,
                    "categoria": categoria,
                    "cidade": "rio" if "rio" in categoria else "paris",
                    "tipo": "roteiros" if "roteiros" in categoria else "logistica"
                }
            }
            
            vectors_to_upsert.append(vector_data)
            print(f"📝 Preparado: {vector_id}")
    
    # Fazer upsert em lote
    print(f"🚀 Inserindo {len(vectors_to_upsert)} vetores...")
    index.upsert(vectors=vectors_to_upsert)
    
    print("✅ Indexação completa!")
else:
    print("✅ Dados já indexados!")

## 6️⃣ **Teste de Conectividade**

In [ ]:
# Aguardar propagação dos dados
time.sleep(2)

# Verificar estatísticas finais
stats = index.describe_index_stats()
print("📊 **ESTATÍSTICAS DO ÍNDICE:**")
print(f"   📈 Total de vetores: {stats['total_vector_count']}")
print(f"   🏷️ Namespaces: {list(stats.get('namespaces', {}).keys()) if stats.get('namespaces') else 'default'}")
print(f"   💾 Dimensão: {stats.get('dimension', 'N/A')}")

In [ ]:
# Teste de busca simples
query = "pontos turísticos famosos"
query_embedding = embeddings.embed_query(query)

# Buscar documentos similares
results = index.query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)

print(f"🔍 **TESTE DE BUSCA:** '{query}'")
print("\n📋 **RESULTADOS:**")
for i, match in enumerate(results['matches'], 1):
    score = match['score']
    text = match['metadata']['text']
    cidade = match['metadata']['cidade']
    
    print(f"\n{i}. **{cidade.upper()}** (Score: {score:.3f})")
    print(f"   {text}")

## ✅ **Resumo do Notebook 01:**

### 🎯 **O que fizemos:**
- ⚙️ Configuramos o Pinecone Cloud
- 🗄️ Criamos índice vetorial "turismo-inteligente"
- 📚 Indexamos 20 documentos turísticos (Rio + Paris)
- 🧠 Usamos embeddings sentence-transformers
- ✅ Testamos busca por similaridade

### 📊 **Dados Indexados:**
- 🇧🇷 **Rio**: 10 docs (5 roteiros + 5 logística)
- 🇫🇷 **Paris**: 10 docs (5 roteiros + 5 logística)
- 📏 **Dimensão**: 384 (all-MiniLM-L6-v2)

### ➡️ **Próximo Passo:**
**02-RAG.ipynb** → Sistema de recuperação inteligente

---
✨ **Base vetorial pronta para uso!**